# IBM Applied Data Science Capstone Course by Coursera

Week 5 Final Report
Opening a New coffee shop in New Delhi,India

1. Build a dataframe of headquaters in new Delhi by web scraping the data from Wikipedia page
2. Get the geographical coordinates of the headquaters.
3. Obtain the venue data for the headquaters from Foursquare API
4. Explore and cluster the headquaters
5. Select the best cluster to open a new coffee shop.


# 1. Importing libraries

In [10]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


# 2. Scrap data from Wikipedia page into a DataFrame

In [31]:
#requests

data = requests.get("https://en.wikipedia.org/wiki/List_of_districts_of_Delhi").text

In [32]:
soup = BeautifulSoup(data, 'html.parser')
# create a list to store neighborhood data
District=[]
Headquarters=[]
SubDivisions=[]


In [33]:
# append the data into the list
for row in soup.find('table').find_all('tr'):
    cellData=row.find_all('td')
    if(len(cellData)>0):
        District.append(cellData[1].text)
        Headquarters.append(cellData[2].text.replace('\n','').strip())
        SubDivisions.append(cellData[3].text.replace('\n','').strip())
        

In [37]:
#Converting to a data frame

delhi_df=pd.DataFrame({"District":District,"Headquarters": Headquarters,"SubDivisions":SubDivisions})
delhi_df.head(20)

,District,Headquarters,SubDivisions
0,New Delhi,Connaught Place,Chanakyapuri
1,North Delhi,Alipur,Model Town[3]
2,North West Delhi,Kanjhawala,Rohini
3,West Delhi,Rajouri Garden,Patel Nagar
4,South West Delhi,Dwarka,Dwarka
5,South Delhi,Saket,Saket
6,South East Delhi,Defence Colony,Defence Colony
7,Central Delhi,Daryaganj,Kotwali
8,North East Delhi,Nand Nagri,Seelampur
9,Shahdara,Shahdara,Shahdara


# 3. Get the geographical coordinates

In [38]:
# define a function to get coordinates
def get_latlng(headquater):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, New Delhi, India'.format(headquater))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [40]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(headquater) for headquater in delhi_df["Headquarters"].tolist() ]
coords

[[28.633920000000046, 77.21978000000007],
 [28.798050000000046, 77.14425000000006],
 [28.70488951402189, 77.00056594180218],
 [28.645620000000065, 77.12209000000007],
 [28.589950000000044, 77.04007000000007],
 [28.524090000000058, 77.20676000000003],
 [28.573000000000036, 77.23360000000008],
 [28.62832000000003, 77.24727000000007],
 [28.696700000000078, 77.30386000000004],
 [28.68957000000006, 77.27802000000008],
 [28.639030000000048, 77.29597000000007]]

In [43]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])


In [57]:
#adding lat lng
columns=["Headquaters","Latitude","Longitude"]
delhi_dfNew1=pd.DataFrame(columns=columns)

for data in delhi_df["Headquarters"]:
    delhi_dfNew1["Headquaters"]=delhi_df["Headquarters"]
    

    

delhi_dfNew1["Latitude"]=df_coords["Latitude"]
delhi_dfNew1["Longitude"]=df_coords["Longitude"]



delhi_dfNew1

,Headquaters,Latitude,Longitude
0,Connaught Place,28.63392,77.219780
1,Alipur,28.79805,77.144250
2,Kanjhawala,28.70489,77.000566
3,Rajouri Garden,28.64562,77.122090
4,Dwarka,28.58995,77.040070
5,Saket,28.52409,77.206760
6,Defence Colony,28.57300,77.233600
7,Daryaganj,28.62832,77.247270
8,Nand Nagri,28.69670,77.303860
9,Shahdara,28.68957,77.278020


In [58]:
# save the DataFrame as CSV file
delhi_dfNew1.to_csv("delhi_dfNew.csv", index=False)

In [59]:
# get the coordinates of Kuala Lumpur
address = 'New Delhi, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Delhi, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New Delhi, India 28.6141793, 77.2022662.


# 4. Create a map of New Delhi with headqauters superimposed on top

In [61]:
# create map of Toronto using latitude and longitude values
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, headquater in zip(delhi_dfNew1['Latitude'], delhi_dfNew1['Longitude'], delhi_dfNew1['Headquaters']):
    label = '{}'.format(headquater)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_delhi)  
    
map_delhi

In [62]:
# save the map as HTML file
map_delhi.save('map_delhi.html')

# 5. Use the Foursquare API to explore the neighborhoods

In [63]:
# define Foursquare Credentials and Version
CLIENT_ID='UBKVQPHZ0N2M23PIWVP4UBKZQAK410PCIT2VABDOOV3G5MEG'
CLIENT_SECRET='Y25V03BWK1PPRZRM4RZSJE55X1RWAOARBWWLWNB4VROPGREI'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UBKVQPHZ0N2M23PIWVP4UBKZQAK410PCIT2VABDOOV3G5MEG
CLIENT_SECRET:Y25V03BWK1PPRZRM4RZSJE55X1RWAOARBWWLWNB4VROPGREI


Now, let's get the top 100 venues that are within a radius of 2000 meters.


In [64]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, headquater in zip(delhi_dfNew1['Latitude'], delhi_dfNew1['Longitude'], delhi_dfNew1['Headquaters']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            headquater,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [65]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Headquaters', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(482, 7)


,Headquaters,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Connaught Place,28.63392,77.21978,Naturals Ice Cream,28.634455,77.222139,Ice Cream Shop
1,Connaught Place,28.63392,77.21978,Connaught Place | कनॉट प्लेस (Connaught Place),28.632731,77.220018,Plaza
2,Connaught Place,28.63392,77.21978,Sagar Ratna,28.635487,77.220650,Indian Restaurant
3,Connaught Place,28.63392,77.21978,Wenger's,28.633412,77.218292,Bakery
4,Connaught Place,28.63392,77.21978,Starbucks,28.632011,77.217731,Coffee Shop


## Let's check how many venues were returned for each neighorhood

In [66]:
venues_df.groupby(["Headquaters"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Headquaters,,,,,,
Alipur,4,4,4,4,4,4
Connaught Place,100,100,100,100,100,100
Daryaganj,41,41,41,41,41,41
Defence Colony,99,99,99,99,99,99
Dwarka,38,38,38,38,38,38
Nand Nagri,6,6,6,6,6,6
Preet Vihar,28,28,28,28,28,28
Rajouri Garden,60,60,60,60,60,60
Saket,100,100,100,100,100,100


In [67]:
#Finding the unique category


print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 111 uniques categories.


In [68]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Ice Cream Shop', 'Plaza', 'Indian Restaurant', 'Bakery',
       'Coffee Shop', 'Café', 'South Indian Restaurant', 'Clothing Store',
       'Molecular Gastronomy Restaurant', 'Food Truck', 'Bar', 'Bistro',
       'Lounge', 'Hotel', 'North Indian Restaurant', 'Deli / Bodega',
       'Gastropub', 'Chinese Restaurant', 'Portuguese Restaurant',
       'BBQ Joint', 'Asian Restaurant', 'Food & Drink Shop',
       'Italian Restaurant', 'Restaurant', 'Tea Room', 'Flea Market',
       'Salon / Barbershop', 'Tibetan Restaurant', 'Beer Garden', 'Spa',
       'Arcade', 'Mediterranean Restaurant', 'Historic Site', 'Theater',
       'Art Gallery', 'Korean Restaurant', 'Indian Chinese Restaurant',
       'Pub', 'Food', 'Fast Food Restaurant', 'Concert Hall',
       'Miscellaneous Shop', 'Snack Place', 'Hardware Store',
       'Performing Arts Venue', 'Sandwich Place', 'Light Rail Station',
       'Breakfast Spot', 'Hostel', 'Bus Station'], dtype=object)

# 6. Analyze Each Neighborhood


In [70]:
# one hot encoding
delhi_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
delhi_onehot['Headquaters'] = venues_df['Headquaters'] 

# move neighborhood column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

print(delhi_onehot.shape)
delhi_onehot.head()

(482, 112)


,Headquaters,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Garden,Bistro,Bookstore,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Electronics Store,English Restaurant,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gift Shop,Golf Course,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Metro Station,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Neighborhood,Nightclub,North Indian Restaurant,Other Great Outdoors,Park,Performing Arts Venue,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Resort,Restaurant,Road,Salon / Barbershop,Sandwich Place,Sculpture Garden,Shopping Mall,Snack Place,South Indian Restaurant,Spa,Stadium,Steakhouse,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Weight Loss Center
0,Connaught Place,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Connaught Place,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Connaught Place,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Connaught Place,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Connaught Place,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [78]:
delhi_grouped1 = delhi_onehot.groupby(["Headquaters"]).mean().reset_index()

print(delhi_grouped1.shape)
delhi_grouped1

(10, 112)


,Headquaters,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beer Garden,Bistro,Bookstore,Breakfast Spot,Burger Joint,Burmese Restaurant,Bus Station,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Electronics Store,English Restaurant,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gift Shop,Golf Course,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Metro Station,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Neighborhood,Nightclub,North Indian Restaurant,Other Great Outdoors,Park,Performing Arts Venue,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Resort,Restaurant,Road,Salon / Barbershop,Sandwich Place,Sculpture Garden,Shopping Mall,Snack Place,South Indian Restaurant,Spa,Stadium,Steakhouse,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Weight Loss Center
0,Alipur,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.25,0.00,0.000000,0.00,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.25,0.250000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,Connaught Place,0.000000,0.010000,0.010000,0.00000,0.020000,0.000000,0.010000,0.020000,0.050000,0.01,0.010000,0.00,0.01,0.000000,0.00,0.00,0.080000,0.030000,0.010000,0.000000,0.040000,0.00,0.01000,0.000000,0.00,0.00000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.010000,0.010000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.00,0.000000,0.00,0.01,0.02000,0.00000,0.00000,0.000000,0.01000,0.120000,0.010000,0.01,0.150000,0.000000,0.010000,0.000000,0.01,0.010000,0.040000,0.000000,0.01,0.000000,0.01,0.000000,0.01,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.01,0.000000,0.000000,0.01,0.000000,0.000000,0.01000,0.00000,0.01,0.000000,0.010000,0.00,0.030000,0.00000,0.01,0.010000,0.000000,0.000000,0.010000,0.020000,0.01,0.000000,0.00,0.010000,0.000000,0.00,0.00,0.010000,0.01,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,Daryaganj,0.000000,0.024390,0.097561,0.04878,0.000000,0.000000,0.000000,0.048780,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.073171,0.000000,0.000000,0.000000,0.024390,0.00,0.02439,0.000000,0.00,0.02439,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.02439,0.04878,0.02439,0.000000,0.04878,0.048780,0.000000,0.00,0.097561,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.024390,0.00,0.02439,0.000000,0.024390,0.00,0.000000,0.000000,0.00,0.000000,0.00000

In [79]:
len(delhi_grouped[delhi_grouped["Coffee Shop"] > 0])

6

## Create a new DataFrame for coffee shop data only

In [80]:
delhi_coffeeShop = delhi_grouped1[["Headquaters","Coffee Shop"]]
delhi_coffeeShop

,Headquaters,Coffee Shop
0,Alipur,0.000000
1,Connaught Place,0.040000
2,Daryaganj,0.024390
3,Defence Colony,0.040404
4,Dwarka,0.000000
5,Nand Nagri,0.000000
6,Preet Vihar,0.035714
7,Rajouri Garden,0.033333
8,Saket,0.090000
9,Shahdara,0.000000


## 7. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in delhi into 3 clusters.

In [81]:
# set number of clusters
kclusters = 3

kl_clustering = delhi_coffeeShop.drop(["Headquaters"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 2, 2, 0, 0, 2, 2, 1, 0])

In [84]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
delhi_merged1 = delhi_coffeeShop.copy()

# add clustering labels
delhi_merged1["Cluster Labels"] = kmeans.labels_
delhi_merged1

,Headquaters,Coffee Shop,Cluster Labels
0,Alipur,0.000000,0
1,Connaught Place,0.040000,2
2,Daryaganj,0.024390,2
3,Defence Colony,0.040404,2
4,Dwarka,0.000000,0
5,Nand Nagri,0.000000,0
6,Preet Vihar,0.035714,2
7,Rajouri Garden,0.033333,2
8,Saket,0.090000,1
9,Shahdara,0.000000,0


In [85]:
#defining the lat lng respectively
delhi_merged1=delhi_merged1.join(delhi_dfNew1.set_index("Headquaters"),on="Headquaters")
delhi_merged1

,Headquaters,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Alipur,0.000000,0,28.79805,77.14425
1,Connaught Place,0.040000,2,28.63392,77.21978
2,Daryaganj,0.024390,2,28.62832,77.24727
3,Defence Colony,0.040404,2,28.57300,77.23360
4,Dwarka,0.000000,0,28.58995,77.04007
5,Nand Nagri,0.000000,0,28.69670,77.30386
6,Preet Vihar,0.035714,2,28.63903,77.29597
7,Rajouri Garden,0.033333,2,28.64562,77.12209
8,Saket,0.090000,1,28.52409,77.20676
9,Shahdara,0.000000,0,28.68957,77.27802


In [86]:
# sort the results by Cluster Labels
print(delhi_merged1.shape)
delhi_merged1.sort_values(["Cluster Labels"], inplace=True)
delhi_merged1

(10, 5)


,Headquaters,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Alipur,0.000000,0,28.79805,77.14425
4,Dwarka,0.000000,0,28.58995,77.04007
5,Nand Nagri,0.000000,0,28.69670,77.30386
9,Shahdara,0.000000,0,28.68957,77.27802
8,Saket,0.090000,1,28.52409,77.20676
1,Connaught Place,0.040000,2,28.63392,77.21978
2,Daryaganj,0.024390,2,28.62832,77.24727
3,Defence Colony,0.040404,2,28.57300,77.23360
6,Preet Vihar,0.035714,2,28.63903,77.29597
7,Rajouri Garden,0.033333,2,28.64562,77.12209


## finally visualize the clusters

In [88]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, head, cluster in zip(delhi_merged1['Latitude'], delhi_merged1['Longitude'],delhi_merged1['Headquaters'], delhi_merged1['Cluster Labels']):
    label = folium.Popup(str(head) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 8. Examine Clusters

## cluster 0

In [89]:
delhi_merged1.loc[delhi_merged1['Cluster Labels'] == 0]

,Headquaters,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Alipur,0.0,0,28.79805,77.14425
4,Dwarka,0.0,0,28.58995,77.04007
5,Nand Nagri,0.0,0,28.69670,77.30386
9,Shahdara,0.0,0,28.68957,77.27802


## cluster 1

In [90]:
delhi_merged1.loc[delhi_merged1['Cluster Labels'] == 1]

,Headquaters,Coffee Shop,Cluster Labels,Latitude,Longitude
8,Saket,0.09,1,28.52409,77.20676


## cluster 2

In [91]:
delhi_merged1.loc[delhi_merged1['Cluster Labels'] == 2]

,Headquaters,Coffee Shop,Cluster Labels,Latitude,Longitude
1,Connaught Place,0.040000,2,28.63392,77.21978
2,Daryaganj,0.024390,2,28.62832,77.24727
3,Defence Colony,0.040404,2,28.57300,77.23360
6,Preet Vihar,0.035714,2,28.63903,77.29597
7,Rajouri Garden,0.033333,2,28.64562,77.12209


# Observations